<a href="https://colab.research.google.com/github/SATHYARANGARAJ53/medicine_inventory/blob/main/projectmodel_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import datetime

def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

# Extract data for the current month from past six years
def extract_relevant_data(df):
    current_month = datetime.datetime.now().strftime('%B_%Y')
    years = [(datetime.datetime.now().year - i) for i in range(1, 7)]

    month_years = [current_month.replace(str(datetime.datetime.now().year), str(year)) for year in years]

    relevant_cols = ['Medicine Name'] + month_years + ['Stock Level']
    df_filtered = df[relevant_cols].dropna()
    return df_filtered, month_years

# Prepare data for LSTM
def prepare_data(df, month_years):
    scaler = MinMaxScaler()
    X = df[month_years].values
    y = df[month_years[-1]].values
    X = scaler.fit_transform(X)
    y = scaler.fit_transform(y.reshape(-1, 1))

    X = X.reshape(X.shape[0], X.shape[1], 1)
    return X, y, scaler

# Define LSTM model
def build_lstm_model():
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(6, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train model and save predictions to a text file
def train_and_predict(file_path, output_file="predictions.txt"):
    df = load_data(file_path)
    df_filtered, month_years = extract_relevant_data(df)
    X, y, scaler = prepare_data(df_filtered, month_years)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = build_lstm_model()

    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1, validation_data=(X_test, y_test))

    predictions = model.predict(X)
    predictions = scaler.inverse_transform(predictions)

    predictions_df = pd.DataFrame(predictions, columns=['Predicted_Consumption'])
    predictions_df['Medicine Name'] = df_filtered['Medicine Name'].reset_index(drop=True)

    predictions_df.to_csv(output_file, sep='\t', index=False)

    print(f"Predictions saved to {output_file}")
    return predictions_df[['Medicine Name', 'Predicted_Consumption']]

file_path = "/content/Medicine_Details.csv"
output_file = "/content/predictions.txt"

predicted_df = train_and_predict(file_path, output_file)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.2926 - val_loss: 0.1753
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2204 - val_loss: 0.1334
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1723 - val_loss: 0.0981
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1644 - val_loss: 0.0704
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0971 - val_loss: 0.0558
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0727 - val_loss: 0.0583
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0824 - val_loss: 0.0666
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0825 - val_loss: 0.0625
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0879 - val_loss: 0.0549
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0867 - val_loss: 0.0519
Epoch 11/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0645 - val_loss: 0.0511
Epoch 12/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0672 - val_loss: 0.0515
Epoch 13/50
